# Gibson On Walkabout

In this notebook, you will handle Gibson's attempts to update the database.

First, run the following cells to set your database credentials, and to reset the database:

## Setting up

The next group of cells set up your database connection, and reset the database to a clean state. Check notebook *8.1 Data Definition Language in SQL* if you are unsure what the next cells do.

You may need to change the given values of the variables `DB_USER` and `DB_PWD`, depending on which environment you are using

In [ ]:
# Set up the PostgreSQL environment

%run sql_init.ipynb
print("Connecting with connection string : {}".format(DB_CONNECTION))
%sql $DB_CONNECTION

In [ ]:
%run reset_databases.ipynb

In [ ]:
import psycopg2 as pg
import psycopg2.extensions as pge

And as in notebook 12.1, we can define a short function, `transaction_status`, which returns a string describing the transaction status of the given connection:

In [ ]:
def transaction_status(conn):
    '''
    Return a string showing the transaction status of the
    given connection, conn.
    '''
    transaction_status_dict={
        pg.extensions.TRANSACTION_STATUS_IDLE:"The session is idle and there is no current transaction.",
        pg.extensions.TRANSACTION_STATUS_ACTIVE:"A command is currently in progress.",
        pg.extensions.TRANSACTION_STATUS_INTRANS:"The session is idle in a valid transaction block.",
        pg.extensions.TRANSACTION_STATUS_INERROR:"The session is idle in a failed transaction block.",
        pg.extensions.TRANSACTION_STATUS_UNKNOWN:"Reported if the connection with the server is bad."
    }
    return transaction_status_dict[conn.get_transaction_status()]
    

## STEP 0
Run the following cells to set up Gibson's connection to the database.

In [ ]:
gibson = pg.connect(dbname=DB_NAME,     # the name of the database
                    host='localhost',   # the host on which the database engine is running
                    user=DB_USER,       # id of the user who is logging in
                    password=DB_PWD,    # the user's password
                    port=5432,          # the port on which the database engine is listening
                    options="-c search_path=hospital")  # the schema to use

gibson.isolation_level = pge.ISOLATION_LEVEL_SERIALIZABLE

#We want to have control over the transaction commit behaviour
gibson.autocommit = False

## STEP 1: Start a Transaction

Run the following cell to start a transaction that involves updating a record and check the transaction status:

In [ ]:
with gibson.cursor() as gibson_cursor:
    
    gibson_cursor.execute('''
        UPDATE patient 
        SET weight_kg = 195
            WHERE patient_id = 'p001';
        ''')
    
print('Gibson:',transaction_status(gibson))

__DO NOT COMMIT THIS TRANSACTION YET.__

__In the `12.1 Paxton.ipynb` notebook, complete STEP 0 and STEP 2 to set up a connection to the database and try to update the same record.__

## STEP 3

Once Paxton has tried to update Thornton's record via STEP 2 in the other notebook, commit Gibson's update:

In [ ]:
gibson.commit()

__Now refer back to Paxton's notebook, STEP 4.__

## STEP 6

Gibson tries to change the record back...

In [ ]:
with gibson.cursor() as gibson_cursor:
    
    gibson_cursor.execute('''
        UPDATE patient 
        SET weight_kg = 125
            WHERE patient_id = 'p001';
        ''')

and the cell execution is blocked.

__Go to STEP 7 in the Paxton notebook, where Gibson will decide to rollback their proposed update. As they do so, watch what happens to the status of the code cell above.__

When Paxton rolls back their attempt to update the record, and the closes their transaction, Gibson's attempt to update the record is accepted.

They can now commit their update.

In [ ]:
gibson.commit()

Gibson makes sure her connection is closed...

In [ ]:
gibson.close()

...then switches off her computer and goes to get a cup of tea...